In [ ]:
# from secret_key import openai_key
import os
os.environ["OPENAI_API_KEY"] = "sk-sjkshdfkjhlsdjhflkjshdljfhsldfljhsjldhflkhsldfgh"

In [ ]:
# %pip install gpt4all

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(
    temperature=0.5
)

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant that serves {cuisine}. What should I name it?"
)

prompt_template_name.format(cuisine="Italian")

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("American")

## *Sequential Chain* 

In [ ]:
llm = OpenAI(temperature=0.5)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant that serves {cuisine}. What should I name it?"
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_name)

prompt_template_name = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it in comma separated format."
)

food_item_chain = LLMChain(llm = llm, prompt = prompt_template_name)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains = [name_chain, food_item_chain]
)
chain.run("Italian")

### Simple Sequential Chain, only returns the final output not the intermediate one

In [ ]:
llm = OpenAI(temperature=0.5)

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to open a restaurant that serves {cuisine}. What should I name it?"
)

name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key='restaurant_name')

prompt_template_name = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu items for {restaurant_name}. Return it in comma separated format."
)

food_item_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key='menu_items')

In [ ]:
from langchain.chains import SequentialChain

SequentialChain(
    chains = [name_chain, food_item_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', 'menu_items']
)

chain({'input': {'cuisine': 'Italian'}})

## LangChain Agents

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools([
    "wikipedia",
    "llm-math"
], llm = llm)

agent = initialize_agent(
    tools, llm, agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = False
)

agent.run("When was Elon Musk born? What is his age right now in 2024")

## Serp API

In [ ]:
os.environ["SERP_API_KEY"] = "sk-sjkshdfkjhlsdjhflkjshdljfhsldfljhsjldhflkhsldfgh"

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm = llm)

agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = False)

agent.run("What was the US GDP in 2022 plus 5?")

## Memory

by default llm chains are state less, and do not have memory

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
chain = LLMChain(llm = llm, prompt = prompt_template_name, memory = memory)
name = chain.run("Italian")
print(name)

but this would keep growing!

Thus we need to restrit the buffer size

In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm = OpenAI(temperature=0.5), memory = memory, chains = [name_chain, food_item_chain])
print(convo.prompt)

by default this object comes with memory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(window_size = 4)

convo = ConversationChain(llm = OpenAI(temperature=0.5), memory = memory, chains = [name_chain, food_item_chain])
print(convo.prompt)